In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
from tqdm import tqdm
import os


from holodeck import plot, detstats, utils
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC, GYR, PC
import holodeck as holo
from holodeck.sams import sam

import hasasia.sim as hsim

In [ ]:
SHAPE = None
NREALS = 500
NFREQS = 40
NLOUDEST = 10
BGL = 1

TOL=0.01
MAXBADS=5

NVARS = 21

NPSRS = 40
NSKIES = 100

ALL_TARGETS = ['gsmf_phi0', 'gsmf_mchar0_log10', 'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_time',  'hard_gamma_inner']

# Count n_ss

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )


In [ ]:
gamma_ssi1 = dsdat[10]['gamma_ssi']
nn_ss1 = detstats.count_n_ss(gamma_ssi1)
print(nn_ss1.shape)
print(holo.utils.stats(nn_ss1))

In [ ]:
print(gamma_ssi1.shape)

In [ ]:
fig, ax = plot.figax(xlabel='Realization', ylabel='Max individual DP_ssi')
xx = np.arange(NREALS)

dpmax = np.max(gamma_ssi1, axis=(0,3))
for pp in [50, 95]:
    med, *conf = np.percentile(dpmax, (50, 50-pp/2, 50+pp/2), axis=1)
    ax.plot(xx, med, color='tab:blue')
    ax.fill_between(xx, *conf, color='tab:blue', alpha=0.2)


In [ ]:
fig, ax = plot.figax(xlabel='Rank Loudest', ylabel='Individual DP_ssi in 0th Freq bin', xscale='linear')
xx = np.arange(NLOUDEST)


colors = ['purple', 'red', 'yellow']
handles = []
labels=[]

for ii,pp in enumerate([50, 95, 99]):
    med, *conf = np.percentile(gamma_ssi1[0], (50, 50-pp/2, 50+pp/2), axis=(0,1,))
    print(med.shape)
    h2, = ax.plot(xx, med, color='k')
    hh = ax.fill_between(xx, *conf, color=colors[ii], alpha=0.2)
    handles.append(hh)
    labels.append(f"{pp}% CI")

handles.append(h2)
labels.append('median')

ax.legend(handles=handles, labels=labels, loc='upper right')


In [ ]:
xx = np.arange(NREALS)
yy = nn_ss1

fig, ax = plot.figax(xlabel='Strain Realization R', xscale='linear',
                     ylabel = '$N_\mathrm{SS}$', yscale='linear')

nshow = 100
for ss in range(nshow):
    h1 = ax.scatter(xx, yy[:,ss], alpha=0.5, s=10, marker='x', label='random')

for pp in [50, 95]:
    lo, hi = np.percentile(yy, (50-pp/2, 50+pp/2), axis=1)
    # ax.fill_between(xx, lo, hi, alpha=0.25, color='tab:blue')

med = np.median(yy, axis=1)
h2 = ax.scatter(xx, med, color='tab:blue', alpha=0.5, s=1, marker='o', label='median')

ax.legend(handles=[h1, h2])
ax.set_title(f'mean parameter model, {plot.PARAM_KEYS[target]}={params[10][target]}')

In [ ]:
fig, ax = plot.figax(xlabel='$N_\mathrm{SS}$', ylabel='Count of All Realizations', xscale='linear')

bins = [-0.5, 0.5, 1.5, 2.5, 3.5]
hist = ax.hist(nn_ss1.flatten(), bins=bins,)
ax.set_xticks([0,1,2,3])
ax.axhline(NREALS*NSKIES, label='Total # of Realizations', color='purple')
ax.legend(loc='upper right')

In [ ]:
fig, ax = plot.figax(xlabel='$N_\mathrm{SS}$', ylabel='Fraction of All Realizations', xscale='linear')

bins = [-0.5, 0.5, 1.5, 2.5, 3.5]
hist = ax.hist(nn_ss1.flatten(), bins=bins, density=True)
ax.set_xticks([0,1,2,3])

In [ ]:
fig, ax = plot.figax(xlabel='$N_\mathrm{SS}$', ylabel='Count of Sky Realizations', xscale='linear')

bins = [-0.5, 0.5, 1.5, 2.5, 3.5]
hist = ax.hist(nn_ss1, bins=bins)

In [ ]:
xx_pars = []
nn_ss = []
for vv in range(NVARS):
    gamma_ssi = dsdat[vv]['gamma_ssi']
    _nn_ss = detstats.count_n_ss(gamma_ssi)
    nn_ss.append(_nn_ss)
    xx_pars.append(params[vv][target])

In [ ]:
fig, ax = plot.figax(xlabel=plot.PARAM_KEYS[target], xscale = 'linear', 
                     ylabel='$N_\mathrm{SS}$', yscale='linear')

xx = np.array(xx_pars)
yy = np.array(nn_ss)
print(f"{xx.shape=}, {yy.shape=}")

colors = ['purple', 'red', 'yellow']
handles = []
labels=[]
for ii, pp in enumerate([50, 95, 99]):
    lo, hi = np.percentile(yy, (50-pp/2, 50+pp/2), axis=(1,2))
    hh = ax.fill_between(xx, lo, hi, alpha=0.25, color=colors[ii])
    handles.append(hh)
    labels.append(f"{pp}% CI")

median = np.median(yy, axis=(1,2))
hh, = ax.plot(xx, median, color='tab:blue')
labels.append('median')
handles.append(hh)

mean = np.mean(yy, axis=(1,2))
hh, = ax.plot(xx, mean, color='k', linestyle='--')
handles.append(hh)
labels.append('mean')
print(handles)
ax.legend(handles=handles, labels=labels, loc='upper left')

In [ ]:
fig, ax = plot.figax(xlabel=plot.PARAM_KEYS[target], xscale = 'linear', 
                     ylabel='$N_\mathrm{SS}$', yscale='log')

xx = np.array(xx_pars)
yy = np.array(nn_ss)
print(f"{xx.shape=}, {yy.shape=}")

colors = ['purple', 'red', 'yellow']
handles = []
labels=[]
for ii, pp in enumerate([50, 95, 99]):
    lo, hi = np.percentile(yy, (50-pp/2, 50+pp/2), axis=(1,2))
    hh = ax.fill_between(xx, lo, hi, alpha=0.25, color=colors[ii])
    handles.append(hh)
    labels.append(f"{pp}% CI")

median = np.median(yy, axis=(1,2))
hh, = ax.plot(xx, median, color='tab:blue')
labels.append('median')
handles.append(hh)

mean = np.mean(yy, axis=(1,2))
hh, = ax.plot(xx, mean, color='k', linestyle='--')
handles.append(hh)
labels.append('mean')
print(handles)
ax.legend(handles=handles, labels=labels, loc='upper left')

all sources N_SS

In [ ]:
title='Randomly Sampled Number of Sources'
ylabel = '$N_\mathrm{SS}$'

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    data, params, dsdat = detstats.get_data(target,
            gw_only=False, red2white=None, red_gamma=None,
            nloudest=10, bgl=1, nexcl=0,
            )
    xx_pars = []
    nn_ss = []
    for vv in range(NVARS):
        gamma_ssi = dsdat[vv]['gamma_ssi']
        _nn_ss = detstats.count_n_ss(gamma_ssi)
        nn_ss.append(_nn_ss)
        xx_pars.append(params[vv][target])

    xx = xx_pars
    yy = nn_ss
    for ii, pp in enumerate([50, 95, 99]):
        lo, hi = np.percentile(yy, (50-pp/2, 50+pp/2), axis=(1,2))
        hh = ax.fill_between(xx, lo, hi, alpha=0.25, color=colors[ii])

        median = np.median(yy, axis=(1,2))
        hh, = ax.plot(xx, median, color='tab:blue')

        mean = np.mean(yy, axis=(1,2))
        hh, = ax.plot(xx, mean, color='k', linestyle='--')

plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)


## Mean Parameter Model dp_2nd vs dp_1st

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )
dsdat_mean = dsdat[10]
data_mean = data[10]

In [ ]:
hss = data_mean['hc_ss'] # F,R,L
hss = np.repeat(hss, NSKIES).reshape(NFREQS, NREALS,NLOUDEST,NSKIES) # F,R,L,S
hss = np.swapaxes(hss, 1,2).reshape(NFREQS*NLOUDEST, NSKIES,NREALS) # F*L, R,S
hss = np.swapaxes(hss, 1,2).reshape(NFREQS*NLOUDEST, NSKIES*NREALS) # F*L, S*R

ssi = dsdat_mean['gamma_ssi'] # F,R,S,L
ssi = np.swapaxes(ssi, 1,3).reshape(NFREQS*NLOUDEST, NSKIES*NREALS) # F*L, S*R
argmax = np.argmax(ssi, axis=0) 
reals = np.arange(NREALS*NSKIES)
dpmax1 = ssi[argmax, reals]
hcmax1 = hss[argmax, reals]
ssi[argmax, reals] = 0

argmax = np.argmax(ssi, axis=0)
dp2nd1 = ssi[argmax, reals]
hc2nd1 = hss[argmax, reals]

In [ ]:
print(dpmax1.shape, dp2nd1.shape)

In [ ]:
fig, ax = plot.figax(xlabel='log DP 1st', ylabel='log DP 2nd', xscale='linear', yscale='linear')
hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(dpmax1), np.log10(dp2nd1), bins=(30,30))
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log$N$')

In [ ]:
fig, axs = plot.figax(nrows=2, ylabel='Count')
bins = np.logspace(-5, 0)

labels = ['DP _1st','DP _2nd' ]
for ii, dp in enumerate([dpmax1, dp2nd1]):
    axs[ii].set_xlabel(labels[ii])
    hist = axs[ii].hist(dp, bins=bins, histtype='step')

fig.subplots_adjust(hspace=0.4)

for ax in axs:
    ax.axhline(NREALS*NSKIES, label='Total # of Realizations', color='purple')
axs[1].legend(loc='upper right')

In [ ]:
fig, ax = plot.figax(xlabel='log DP_1st', ylabel = 'log hc(DP_1st)', 
                     xscale='linear', yscale='linear')
hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(dpmax1), np.log10(hcmax1), bins = (40,40))
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log$N$')


In [ ]:
fig, ax = plot.figax(xlabel='log DP_2nd', ylabel = 'log hc(DP_2nd)', 
                     xscale='linear', yscale='linear')
hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(dp2nd1), np.log10(hc2nd1), bins = (40,40))
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log$N$')


In [ ]:
fig, ax = plot.figax(xlabel='log DP_1std', ylabel = 'log [$h_c$(DP_1st)/$h_c$(DP_2nd)]', 
                     xscale='linear', yscale='linear')
hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(dpmax1), np.log10(hcmax1/hc2nd1), bins = (40,40))
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log$N$')


In [ ]:
fig, ax = plot.figax(xlabel='log $h_c$(DP_1st)', ylabel = 'log [$h_c$(DP_2nd)]', 
                     xscale='linear', yscale='linear')
hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(hcmax1), np.log10(hc2nd1), bins = (40,40), weights=dpmax1)
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log DP1st-weighted $N$')


In [ ]:
hss = data_mean['hc_ss'][0] # R,L
hss = np.repeat(hss, NSKIES).reshape(NREALS,NLOUDEST,NSKIES) # R,L,S
hss = np.swapaxes(hss, 0,1).reshape(NLOUDEST, NREALS, NSKIES) # L, R,S
hss = np.swapaxes(hss, 1,2).reshape(NLOUDEST, NSKIES*NREALS) # L, S*R

ssi = dsdat_mean['gamma_ssi'][0] # R,S,L
ssi = np.swapaxes(ssi, 0,2).reshape(NLOUDEST, NSKIES*NREALS) # F*L, S*R
argmax = np.argmax(ssi, axis=0) 
reals = np.arange(NREALS*NSKIES)
dpmax1 = ssi[argmax, reals]
hcmax1 = hss[argmax, reals]
ssi[argmax, reals] = 0

argmax = np.argmax(ssi, axis=0)
dp2nd1 = ssi[argmax, reals]
hc2nd1 = hss[argmax, reals]

In [ ]:
fig, ax = plot.figax(xlabel='log DP_1st', ylabel = 'log hc(DP_1st)', 
                     xscale='linear', yscale='linear')
ax.set_title('1st Freq Bin')
hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(dpmax1), np.log10(hcmax1), bins = (40,40))
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log$N$')


In [ ]:
fig, ax = plot.figax(xlabel='log DP_2nd', ylabel = 'log hc(DP_2nd)', 
                     xscale='linear', yscale='linear')
ax.set_title('1st Freq Bin')

hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(dp2nd1), np.log10(hc2nd1), bins = (40,40))
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log$N$')


In [ ]:
fig, ax = plot.figax(xlabel='log DP_1std', ylabel = 'log [$h_c$(DP_1st)/$h_c$(DP_2nd)]', 
                     xscale='linear', yscale='linear')
ax.set_title('1st Freq Bin')
hist2d, xx_edges, yy_edges = np.histogram2d(np.log10(dpmax1), np.log10(hcmax1/hc2nd1), bins = (40,40))
hist2d = hist2d.T # transpose to follow cartesian
xx_grid, yy_grid = np.meshgrid(xx_edges, yy_edges)
im = ax.pcolormesh(xx_grid, yy_grid, np.log10(hist2d))
cbar = plt.colorbar(im, ax=ax, orientation='vertical', label='log$N$')


# Plot Number of Sources Contributing to EV_SS

In [ ]:
BUILD_ARRAYS=False

ylabel='$N(\mathrm{DP_{SS}}>x)$'
title = 'Number of Sources Contributing to $\langle N \\rangle_\mathrm{SS}$'

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', yscale='linear', height=7)
fig.text(0.08, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, y1, y2, _, _, _, _ = detstats.build_noise_arrays(target)
    else:
        _, y1, y2, _, _, _, _ = detstats.get_noise_arrays_temp(target)

    y1 = y1.reshape(NVARS, NSKIES*NREALS)
    y2 = y2.reshape(NVARS, NREALS*NSKIES)

    h1 = plot.draw_med_conf_color(ax, xx, y1, color='tab:orange')
    h2 = plot.draw_med_conf_color(ax, xx, y2, color='tab:blue')

plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['$0.50$', '$0.01$']
fig.legend(handles=[h1, h2], labels=labels, bbox_to_anchor=(0.5,-0.02), loc='lower center',
           ncols=2, title='$x$ threshold')

### with n_excl=3: hard_time

In [ ]:
# load data

nexcl=3
target='hard_time'
white_noise, y1, y2, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target, nexcl=nexcl, save_temp=False)


In [ ]:
# plot data 
ylabel='$N(\mathrm{DP_{SS}}>x)$'
title = 'Number of Sources Contributing to $\langle N \\rangle_\mathrm{SS}$'

fig, ax = plot.figax_single(
                              xscale='linear', yscale='linear',)
ax.set_ylabel(ylabel)
ax.set_title(title, fontsize=12)
ax.set_xlabel(plot.PARAM_KEYS[target])

xx, _ = detstats.get_ratio_arrays(target)

y1 = y1.reshape(NVARS, NSKIES*NREALS)
y2 = y2.reshape(NVARS, NREALS*NSKIES)

h1 = plot.draw_med_conf_color(ax, xx, y1, color='tab:orange')
h2 = plot.draw_med_conf_color(ax, xx, y2, color='tab:blue')


labels = ['$0.50$', '$0.01$']

ax.legend(handles=[h1, h2], labels=labels, loc='upper left',
           ncols=2, title='$x$ threshold')


############################# 2nd plot #########################

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0
fobs, _ = holo.utils.pta_freqs()

fig2, ax2 = plot.figax_single(xscale='linear')
ax2.set_ylabel(ylabel)
ax2.set_title(title, fontsize=12)
psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                        reshape_freqs=False) # V,F,R,L

yy_wn = white_noise.reshape(NVARS, NREALS)
yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

h1 = plot.draw_med_conf_color(ax2, xx, yy_wn, color='grey', linestyle=':')
h2 = plot.draw_med_conf_color(ax2, xx, yy_bg, color='tab:blue', linestyle='--')
h3 = plot.draw_med_conf_color(ax2, xx, yy_ss, color='black', fracs=[0.5,])
h4 = plot.draw_med_conf_color(ax2, xx, yy_l1, color='#00ff00')
h5 = plot.draw_med_conf_color(ax2, xx, yy_l2, color='#ff0000' )
h6 = plot.draw_med_conf_color(ax2, xx, yy_lt, color='#4000ff')
labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
ax2.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=3, columnspacing=1.5)

### with n_excl=3: gsmf_phi0

In [ ]:
# load data

nexcl=3
target='gsmf_phi0'
white_noise, y1, y2, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target, nexcl=nexcl, save_temp=False)


In [ ]:
# plot data 
ylabel='$N(\mathrm{DP_{SS}}>x)$'
title = 'Number of Sources Contributing to $\langle N \\rangle_\mathrm{SS}$'

fig, ax = plot.figax_single(
                              xscale='linear', yscale='linear',)
ax.set_ylabel(ylabel)
ax.set_title(title, fontsize=12)
ax.set_xlabel(plot.PARAM_KEYS[target])

xx, _ = detstats.get_ratio_arrays(target)

y1 = y1.reshape(NVARS, NSKIES*NREALS)
y2 = y2.reshape(NVARS, NREALS*NSKIES)

h1 = plot.draw_med_conf_color(ax, xx, y1, color='tab:orange')
h2 = plot.draw_med_conf_color(ax, xx, y2, color='tab:blue')


labels = ['$0.50$', '$0.01$']

ax.legend(handles=[h1, h2], labels=labels, loc='upper left',
           ncols=2, title='$x$ threshold')


############################# 2nd plot #########################

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0
fobs, _ = holo.utils.pta_freqs()

fig2, ax2 = plot.figax_single(xscale='linear')
ax2.set_ylabel(ylabel)
ax2.set_title(title, fontsize=12)
psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                        reshape_freqs=False) # V,F,R,L

yy_wn = white_noise.reshape(NVARS, NREALS)
yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

h1 = plot.draw_med_conf_color(ax2, xx, yy_wn, color='grey', linestyle=':')
h2 = plot.draw_med_conf_color(ax2, xx, yy_bg, color='tab:blue', linestyle='--')
h3 = plot.draw_med_conf_color(ax2, xx, yy_ss, color='black', fracs=[0.5,])
h4 = plot.draw_med_conf_color(ax2, xx, yy_l1, color='#00ff00')
h5 = plot.draw_med_conf_color(ax2, xx, yy_l2, color='#ff0000' )
h6 = plot.draw_med_conf_color(ax2, xx, yy_lt, color='#4000ff')
labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
ax2.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=3, columnspacing=1.5)

# Plot Noise

In [ ]:
fobs, _ = holo.utils.pta_freqs()

In [ ]:
BUILD_ARRAYS=False

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target)
    else:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.get_noise_arrays_temp(target)

    psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
    psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

    yy_wn = white_noise.reshape(NVARS, NREALS)
    yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
    yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

    yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
    yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
    yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

    h1 = plot.draw_med_conf_color(ax, xx, yy_wn, color='grey', linestyle=':')
    h2 = plot.draw_med_conf_color(ax, xx, yy_bg, color='tab:blue', linestyle='--')
    h3 = plot.draw_med_conf_color(ax, xx, yy_ss, color='black', fracs=[0.5,])
    h4 = plot.draw_med_conf_color(ax, xx, yy_l1, color='#00ff00')
    h5 = plot.draw_med_conf_color(ax, xx, yy_l2, color='#ff0000' )
    h6 = plot.draw_med_conf_color(ax, xx, yy_lt, color='#4000ff')



plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
fig.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Power Spectral Density [Hz$^{-3}$]'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target)
    else:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.get_noise_arrays_temp(target)

    psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
    psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

    yy_wn = white_noise.reshape(NVARS, NREALS)
    yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
    yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

    yy_l1 = psd_ss[:,ff,:,0].reshape(NVARS, NREALS)
    yy_l2 = psd_ss[:,ff,:,1].reshape(NVARS, NREALS)
    yy_lt = psd_ss[:,ff,:,9].reshape(NVARS, NREALS)

    h1 = plot.draw_med_conf_color(ax, xx, yy_wn, color='grey', linestyle=':')
    h2 = plot.draw_med_conf_color(ax, xx, yy_bg, color='tab:blue', linestyle='--')
    h3 = plot.draw_med_conf_color(ax, xx, yy_ss, color='black', fracs=[0.5,])
    h4 = plot.draw_med_conf_color(ax, xx, yy_l1, color='#00ff00')
    h5 = plot.draw_med_conf_color(ax, xx, yy_l2, color='#ff0000' )
    h6 = plot.draw_med_conf_color(ax, xx, yy_lt, color='#4000ff')



plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['WN', 'BG', 'SS', '1st Loudest', '2nd L', '10th L']
fig.legend(handles=[h1, h2, h3, h4, h5, h6], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='Ratio of Power Spectral Density $S_h=h_c^2/(12 \pi^2 f^3)$ in the lowest frequency bin'
ylabel = 'Ratio of Power Spectral Density [Hz$^{-3}$/Hz$^{-3}$]'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.build_noise_arrays(target)
    else:
        white_noise, _, _, hc_ss, hc_bg, _, _ = detstats.get_noise_arrays_temp(target)

    psd_bg = detstats._power_spectral_density(hc_bg, fobs[np.newaxis,:,np.newaxis],
                                          reshape_freqs=False) # V,F,R
    psd_ss = detstats._power_spectral_density(hc_ss, fobs[np.newaxis,:,np.newaxis,np.newaxis],
                                          reshape_freqs=False) # V,F,R,L

    yy_wn = white_noise.reshape(NVARS, NREALS)
    yy_bg = psd_bg[:,ff].reshape(NVARS, NREALS)
    yy_ss = np.sum(psd_ss[:,ff,:,:], axis=-1).reshape(NVARS, NREALS)

    yy_l1l2 = (psd_ss[:,ff,:,0]/psd_ss[:,ff,:,1]).reshape(NVARS, NREALS)
    yy_l1lt = (psd_ss[:,ff,:,0]/psd_ss[:,ff,:,9]).reshape(NVARS, NREALS)

    # h1 = plot.draw_med_conf_color(ax, xx, yy_wn, color='grey', linestyle=':')
    # h2 = plot.draw_med_conf_color(ax, xx, yy_bg, color='tab:blue', linestyle='--')
    # h3 = plot.draw_med_conf_color(ax, xx, yy_ss, color='black', fracs=[0.5,])
    h5 = plot.draw_med_conf_color(ax, xx, yy_l1lt, color='tab:blue', )
    h4 = plot.draw_med_conf_color(ax, xx, yy_l1l2, color='#4000ff', )
    


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

labels = ['1st/2nd L', '1st/10th L']
fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
           ncols=6, columnspacing=1.5)

# Look at rank of loudest vs rank of DP

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )
hc_ss = data[10]['hc_ss']
gamma_ssi = dsdat[10]['gamma_ssi']

In [ ]:
print(gamma_ssi.shape)

In [ ]:
ff=0 # just look at lowest frequency bin
argmax = np.argmax(gamma_ssi[ff], axis=(-1))
argmax += 1
print(f"{argmax.shape=}, {np.max(argmax)=} should = 10")
fig, ax = plot.figax(xlabel='Rank of Loudest', ylabel='Number of Most Detectable',
                     xscale='linear')
bins = np.arange(0,11, dtype=float)
print(bins)
bins += 0.5

ax.hist(np.swapaxes(argmax,0,1), bins)
ax.set_xticks([1,2,3,4,5,6,7,8,9,10])

In [ ]:
ff=0 # just look at lowest frequency bin
argmax = np.argmax(gamma_ssi[ff], axis=(-1))
argmax += 1
print(f"{argmax.shape=}, {np.max(argmax)=} should = 10")
fig, ax = plot.figax(xlabel='Rank of Loudest', ylabel='Number of Most Detectable',
                     xscale='linear')
bins = np.arange(0,11, dtype=float)
print(bins)
bins += 0.5

ax.hist(argmax.flatten(), bins, density=True,)
ax.set_xticks([1,2,3,4,5,6,7,8,9,10])

# Look at dp_ssi comparison

In [ ]:
BUILD_ARRAYS=True

title='Ratio of detection probabilities'
ylabel = 'DP_1st/DP_2nd'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel(plot.PARAM_KEYS[target])

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)

   
    yy = (dp_l1/dp_l2).reshape(NVARS, NREALS*NSKIES)

    h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
  


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='Ratio of detection probabilities'
ylabel = 'DP_1st/DP_2nd'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              xscale='linear', height=7)
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel('DP_1st')

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)

   
    # sort = np.argsort(dp_l1.reshape(NVARS, NREALS*NSKIES))
    # print(sort.shape)
    # yy = (dp_l1/dp_l2).reshape(NVARS, NREALS*NSKIES)
    # print(yy.shape)
    # yy=yy[sort]
    # xx = dp_l1[sort]

    # h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
    # ax.hist2d(dp_l1.flatten(), (dp_l1/dp_l2).flatten(), bins=10)
    ax.scatter(dp_l1.flatten(), (dp_l1/dp_l2).flatten(), alpha=0.1, s=1)


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='2d histogram, log(N)'
ylabel = 'log(DP_1st/DP_2nd)'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              height=7, xscale='linear', yscale='linear')
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    ax.set_xlabel('log(DP_1st)')

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)

   
    # sort = np.argsort(dp_l1.reshape(NVARS, NREALS*NSKIES))
    # print(sort.shape)
    # yy = (dp_l1/dp_l2).reshape(NVARS, NREALS*NSKIES)
    # print(yy.shape)
    # yy=yy[sort]
    # xx = dp_l1[sort]

    # h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
    hist, dprat_edges, dp1st_edges,  = np.histogram2d(np.log10(dp_l1/dp_l2).flatten(), np.log10(dp_l1).flatten(), bins=60)
    dp1st_grid, dprat_grid  = np.meshgrid(dp1st_edges, dprat_edges)
    im = ax.pcolormesh(dp1st_grid, dprat_grid, np.log10(hist))


plt.subplots_adjust(wspace=0.05, hspace=0.38, top=0.93)

# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='2d histogram, log(N)'
ylabel = 'log(DP_2nd)'
xlabel = 'log(DP_1st)'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              height=7, xscale='linear', yscale='linear')
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.text(0.5, 0.08, xlabel, ha='center', va='center')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)



    # h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
    dp2nd_edges = np.linspace(np.log10(np.min(dp_l2)), 
                               np.log10(np.max(dp_l2)), 100)
    dp1st_edges = np.linspace(np.log10(np.min(dp_l1)), 
                               np.log10(np.max(dp_l1)), 200)
    hist, dp2e, dp1e,  = np.histogram2d(np.log10(dp_l2).flatten(), np.log10(dp_l1).flatten(), 
                                                      bins=(dp2nd_edges, dp1st_edges))
    dp1st_grid, dp2nd_grid  = np.meshgrid(dp1st_edges, dp2nd_edges)
    im = ax.pcolormesh(dp1st_grid, dp2nd_grid, np.log10(hist))
    cbar = plt.colorbar(im ,ax=ax, )
    ax.set_xlim(-1, 0)
    ax.set_ylim(-1,0)


plt.subplots_adjust(wspace=0.05, hspace=0.2, top=0.93)
# plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.4, 
#              orientation='horizontal', pad=0.22, label='$\log N$', )



# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

In [ ]:
BUILD_ARRAYS=False

title='2d histogram, log(N)'
ylabel = 'log(DP_2nd)'
xlabel = 'log(DP_1st)'
ff=0

fig, axs = plot.figax_single(nrows=3, ncols=2, sharey=True, sharex=False,
                              height=7, xscale='linear', yscale='linear')
fig.text(0.02, 0.5, ylabel, ha='center', va='center', rotation='vertical')
fig.text(0.5, 0.08, xlabel, ha='center', va='center')
fig.suptitle(title, fontsize=12)
for tt,target in enumerate(ALL_TARGETS):
    ax = axs.flatten()[tt]
    

    xx, _ = detstats.get_ratio_arrays(target)
    if BUILD_ARRAYS:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.build_noise_arrays(target)
    else:
        _, _, _, _, _, dp_l1, dp_l2 = detstats.get_noise_arrays_temp(target)



    # h1 = plot.draw_med_conf_color(ax, xx, yy, color='tab:blue', linestyle='-')
    dp2nd_edges = np.linspace(np.log10(np.min(dp_l2)), 
                               np.log10(np.max(dp_l2)), 100)
    dp1st_edges = np.linspace(np.log10(np.min(dp_l1)), 
                               np.log10(np.max(dp_l1)), 200)
    hist, dp2e, dp1e,  = np.histogram2d(np.log10(dp_l2).flatten(), np.log10(dp_l1).flatten(), 
                                                      bins=(dp2nd_edges, dp1st_edges))
    dp1st_grid, dp2nd_grid  = np.meshgrid(dp1st_edges, dp2nd_edges)
    im = ax.pcolormesh(dp1st_grid, dp2nd_grid, np.log10(hist))
    cbar = plt.colorbar(im ,ax=ax, )
    # ax.set_xlim(-1, 0)
    # ax.set_ylim(-1,0)


plt.subplots_adjust(wspace=0.05, hspace=0.2, top=0.93)
# plt.colorbar(im, ax=axs.ravel().tolist(), shrink=0.4, 
#              orientation='horizontal', pad=0.22, label='$\log N$', )



# labels = ['1st/2nd L', '1st/10th L']
# fig.legend(handles=[ h4, h5], labels=labels, bbox_to_anchor=(0.5,0.0), loc='lower center',
#            ncols=6, columnspacing=1.5)

# Check EV_SS for hard_time

In [ ]:
target = 'hard_time'
data, params, dsdat = detstats.get_data(target,
        gw_only=False, red2white=None, red_gamma=None,
        nloudest=10, bgl=1, nexcl=0,
        )

In [ ]:
logevss = []
xx = []
for ii in range(21):
    logevss.append(np.log10(dsdat[ii]['ev_ss']))
    xx.append(params[ii][target])
logevss = np.array(logevss)
print(logevss.shape)

In [ ]:
fig, ax = plot.figax(xscale='linear', yscale='linear',
                     xlabel=plot.PARAM_KEYS[target],
                     ylabel='$\log{\langle N_\mathrm{SS} \\rangle}$')
yy = logevss.reshape(21, 500*100)
for pp in [50, 95]:
    percs = [50-pp/2, 50+pp/2]
    med, *conf = np.percentile(yy, (50, *percs), axis=1)
    ax.plot(xx, med)
    ax.fill_between(xx, *conf, alpha=0.15)

In [ ]:
dp_ssi = dsdat[10]['gamma_ssi']

In [ ]:
print(dp_ssi.shape)

figure out fancy indexing

In [ ]:
randoms = np.asarray(np.random.rand(3*10).reshape(3,10)*10, dtype=int)
print(randoms)

In [ ]:
rand_argmax = np.argmax(randoms, axis=0, keepdims=False)
print(np.max(rand_argmax))
print(rand_argmax.shape)
print(rand_argmax)

In [ ]:
reals = np.arange(10)

In [ ]:
print(randoms[rand_argmax,reals])

In [ ]:
randoms[rand_argmax,reals] = 0
print(randoms)

In [ ]:
reals = np.arange(100)
index = np.array([rand_argmax,reals])
print(index.shape)

print(randoms[np.swapaxes(index, 0,1)])


In [ ]:
print(randoms[rand_argmax].shape)

In [ ]:

reals = np.arange(NREALS*NSKIES)
dp_max = dp_ssi[argmax, reals]


In [ ]:
dp_ssi = np.swapaxes(dp_ssi, 1,3)
print(dp_ssi.shape)
dp_ssi = dp_ssi.reshape(NFREQS*NLOUDEST, NSKIES*NREALS)
print(dp_ssi.shape)

In [ ]:
argmax = np.argmax(dp_ssi, axis=0, keepdims=True)
print(argmax.shape)